In [1]:
import numpy as np
import igl
import meshplot as mp
import math
import scipy.sparse as sp
from numpy import matlib as mb
from math import sqrt

# Load Data

In [2]:
v, f = igl.read_triangle_mesh('../data/camel_head.off')

kdmin, kdmax, kmin, kmax = igl.principal_curvature(v,f)  
kdmax /= np.linalg.norm(kdmax)
kdmin /= np.linalg.norm(kdmin)
mp.plot(v, f, c=kmax)
mp.plot(v, f, c=kmin)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

In [3]:
def getAdjacentFaces(v,f):
    VF, NI = igl.vertex_triangle_adjacency(f, v.shape[0])
    VFi = []
    for i in range(NI.shape[0] - 1):
        VFii = []
        jj = NI[i + 1] - NI[i]
        for j in range(jj):
            VFii.append(VF[NI[i] + j])
        VFi.append(VFii)
    return VFi

# Compute Principal Curvatures

In [4]:
def principal_curvatures(v,f):
    N = igl.per_vertex_normals(v,f)
    Nf = igl.per_face_normals(v,f,np.array([1,1,1])/np.linalg.norm(np.array([1,1,1])))
    VF, NI = igl.vertex_triangle_adjacency(f, v.shape[0])
    VFi = getAdjacentFaces(v,f)
    TT, TTi = igl.triangle_triangle_adjacency(f)
    A = igl.doublearea(v,f)
    kmin = np.zeros((v.shape[0],1))
    kmax = np.zeros((v.shape[0],1))
    kdmin = np.zeros((v.shape[0],3))
    kdmax = np.zeros((v.shape[0],3))
    
    for v, vi in enumerate(v):
        S = np.zeros((3,3))
        for i, vfi in enumerate(VFi[v]):
            f1 = vfi
            f2 = TT[f1,VFi[v,i]]
            if f2 < 0: # boundary edges
                continue
            vn = f[f1,(VFi[v,i] + 1)%3]
            e = vi - v[vn]
            ne = N[v] + N[vn]
            ne /= np.linalg.norm(ne)
            n1 = Nf[f1]
            n2 = Nf[f2]
            h = 2.0 * np.linalg.norm(e) * np.sqrt((1-np.dot(n1,n2))/2)
            coeff = 0.5 * np.dot(ne,N[v])
            S += (coeff * h * np.cross(e,ne) * np.cross(e,ne)).T
            evalue, evector = np.linalg.eig(S)
            print(evalue)
principal_curvatures(v,f)

TypeError: list indices must be integers or slices, not tuple

# Find Regular Triangle

In [5]:
def sgn(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    return 0

def mark_regular(f,kdmax,kdmin):
    singular = []
    regular = []
    color = []
    for indfi, fi in enumerate(f):
        sign_min = 1
        sign_max = 1
        for i in range(1,3):
            if np.dot(kdmax[fi[0]],kdmax[fi[i]]) < 0:
                kdmax[fi[i]] = - kdmax[fi[i]]
            if np.dot(kdmin[fi[0]],kdmin[fi[i]]) < 0:
                kdmin[fi[i]] = - kdmin[fi[i]]
#         for i in range(3):
#             for j in range(i+1,3):
#                 sign_min *= np.dot(kdmin[fi[i]],kdmin[fi[j]])
#                 sign_max *= np.dot(kdmax[fi[i]],kdmax[fi[j]])
#         if sign_min > 0 and sign_max > 0:
        if np.dot(kdmax[fi[1]],kdmax[fi[2]]) > 0 and np.dot(kdmin[fi[1]],kdmin[fi[2]]) >0:
            regular.append(indfi)
            color.append([1,1,1])
        else:
            singular.append(indfi)
            color.append([0.8,0.8,0.8])
    return np.array(regular), np.array(singular), np.array(color)

regular, singular,color = mark_regular(f,kdmax,kdmin)

# Compute Extremalities

In [37]:
def extremalities(v, f, k, kd):
    EX = np.zeros((v.shape[0],1))
    A = igl.doublearea(v,f)
    G = igl.grad(v, f) # x_f0,x_f1,x_f2 ... y_f0,y_f1,y_f2 ... z_f0,z_f1,z_f2 ...
    grad_k = G * k
    neighbour = getAdjacentFaces(v,f) 
    for indvi, vi in enumerate(v):
        area = 0
        ex = 0
        # vfi: T, indvi: p
        for indvfi, vfi in enumerate(neighbour[indvi]):
            area += A[vfi] # compute area star
        for indvfi, vfi in enumerate(neighbour[indvi]):
            g = np.array([grad_k[vfi], grad_k[f.shape[0] + vfi], grad_k[2 * f.shape[0] + vfi]])
            ex += A[vfi] * np.dot(g,kd[indvi])
        ex /= area
        EX[indvi] = ex
    return EX

EX_max = extremalities(v, f, kmax, kdmax)
EX_min = extremalities(v, f, kmin, kdmin)
mp.plot(v, f, c=EX_min)
mp.plot(v, f, c=EX_max)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

# Smooth Extremalities

In [38]:
def smooth_extremalities(v, f, kd, ex):
    A = igl.adjacency_list(f)
    L = igl.cotmatrix(v,f)
    M = igl.massmatrix(v,f)
    L = M.T * L 
    LEx = np.zeros((v.shape[0],1))
    for indvi, vi in enumerate(v):
        lp = 0
        for vn in A[indvi]:
            sign = sgn(np.dot(kd[indvi],kd[vn]))
            lp += L[indvi,vn] * (sign * ex[vn] - ex[indvi])
        LEx[indvi] = lp
    ex += 0.5*LEx
    return ex

EX_max = smooth_extremalities(v, f, kmax, EX_max)
EX_min = smooth_extremalities(v, f, kmin, EX_min)
mp.plot(v, f, c=EX_min)
mp.plot(v, f, c=EX_max)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

# Extract Feature Line

In [39]:
def extract_feature_line(v, f, kmax, kmin, KD, EX, regular, singular, sign, T):
    # for regular faces
    fr = np.array([f[index] for index in regular])
    TT, TTi = igl.triangle_triangle_adjacency(fr) # id of the adjacent triangle, id of the shared edge
    edges = [[],[]]
    marked_edges = []
    zero_count = [0,0]
    zero_point_id = [[],[]]
    zero_point_v = [[],[]]
    zero_point_k = [[],[]]
    
#     G_main = igl.grad(v,fr)
#     GEX = G_main * EX
#     print(GEX.shape)
#     print(fr.shape)
    # for regular triangles
    for indfri, fri in enumerate(fr):
        ex = np.array([EX[index] for index in fri]) # each vertex's extremalities on the current triangle
        kd = np.array([KD[index] for index in fri]) # each vertex's curvature direction on the current triangle
        # flip signs
        for i in range(1,3):
            if np.dot(kd[0],kd[i]) < 0:
                kd[i] = - kd[i]
                ex[i] = - ex[i]

#         if np.any(ex):
#             continue # skip zeros
        if ex[ex.argmax()] <= 0.0 or ex[ex.argmin()] >= 0.0:
            continue # no zero points
        kd_sum = kd.sum(axis=0)
        vv = np.array([v[index] for index in fri]) # the vertex on this triangle
        G = igl.grad(vv, np.array([[0,1,2]]))
        gex = G * ex # trangle based gradient
#         print(gex)
#         ind = indfri
#         gex = np.array([GEX[ind][0],GEX[fr.shape[0] + ind][0],GEX[2 * fr.shape[0] + ind][0]])
        # check equation (5)
        if sign * np.dot(np.array([kd_sum]),gex) >= 0: 
#         if sign * np.dot(np.array([gex[0][0],gex[1][0],gex[2][0]]), np.array([kd_sum[0],kd_sum[1],kd_sum[2]])) >= 0: 
#         if sign * np.dot(gex, np.array([kd_sum[0],kd_sum[1],kd_sum[2]])) >= 0: 
            continue
            
        # check equation (6)
        kmax_ = np.array([kmax[index] for index in fri])
        kmin_ = np.array([kmin[index] for index in fri])
        if sign * (abs(kmax_.sum(axis=0)) - abs(kmin_.sum(axis=0))) <= 0:
            continue
        count = 0
        for i in range(3):
            j = (i + 1) % 3
            if ex[i] * ex[j] <= 0: # has sign change: should have a zero point in the middle
                # add mid point
                a = abs(ex[i])
                b = abs(ex[j])
                mid = (b * v[fri[i]] + a * v[fri[j]]) / (a + b)
                edges[count].append(mid)
                zero_point_id[count].append(zero_count[count])
                if sign == 1:
                    zero_point_k = (b * kmax[fri[i]] + a * kmax[fri[j]]) / (a + b)
                else:
                    zero_point_k = (b * kmin[fri[i]] + a * kmin[fri[j]]) / (a + b)
                zero_point_v[count].append(mid)
                zero_count[count] += 1
                # mark edge i, j
                marked_edges.append([fri[i],fri[j]])
                count = (count + 1)%2
         
    # for singular triangles           
    for indfi, fi in enumerate(singular):
        face = f[fi]
        marked = []
        for es in range(3): # edge start 0-1, 1-2, 2-0
            ee = (es + 1) % 3 # edge end
            # see whether there are marked edges
            if [face[es],face[ee]] in marked_edges or [face[ee],face[es]] in marked_edges:
                marked.append([face[es],face[ee]])
        if len(marked) < 2:
            continue # one marked edge: do nothing
        if len(marked) == 2:
            for k in range(2):
                i = marked[k][0]
                j = marked[k][1]
                a = abs(EX[i])
                b = abs(EX[j])
                mid = (b * v[i] + a * v[j]) / (a + b)
                edges[k].append(mid)
                
        if len(marked) == 3:
            v0 = v[face][0]
            v1 = v[face][1]
            v2 = v[face][2]
            bc = (v0 + v1 + v2)/3 # barycenter
            for k in range(3):
                edges[0].append(bc)
            for k in range(3):
                i = marked[k][0]
                j = marked[k][1]
                a = abs(EX[i])
                b = abs(EX[j])
                mid = (b * v[i] + a * v[j]) / (a + b)
                edges[1].append(mid)
                

    # create start points and end points
    edges_start = np.zeros((len(edges[0]),3))
    edges_end = np.zeros((len(edges[1]),3))
    for inde, ei in enumerate(edges[0]):
        edges_start[inde] = ei
        edges_end[inde] = edges[1][inde]
    
    return edges_start, edges_end

start_max, end_max = extract_feature_line(v,f,kmax,kmin,kdmax,EX_max,regular, singular, 1, 0.1)
start_min, end_min = extract_feature_line(v,f,kmax,kmin,kdmin,EX_min,regular, singular, -1, 0.1)

# Draw Feature Lines

In [14]:
p = mp.plot(v, f, c=color)
p.add_lines(start_max, end_max, shading={"line_color": "red"})
p.add_lines(start_min, end_min, shading={"line_color": "blue"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

2

In [40]:
p = mp.plot(v, f, c=color)
p.add_lines(start_max, end_max, shading={"line_color": "red"})
p.add_lines(start_min, end_min, shading={"line_color": "blue"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

2

# Smooth Feature Lines

In [41]:
print(start_max)

[[-26.42611593  -4.9308257    3.64924252]
 [-26.50616894  -5.32679301   3.36945107]
 [-26.50616894  -5.32679301   3.36945107]
 ...
 [ -5.42223     -0.32419867   9.74687667]
 [ -5.42223     -0.32419867   9.74687667]
 [ 26.95710988  -3.81600916  11.04237902]]


In [42]:
print(end_max)

[[-26.37453576  -5.12445443   3.561743  ]
 [-26.50826379  -5.33355582   3.36233671]
 [-26.37453576  -5.12445443   3.561743  ]
 ...
 [ -5.78616523  -0.20145839   9.642032  ]
 [ -5.73508344  -0.27451032   9.66882385]
 [ 26.94751918  -3.83559626  11.04091753]]
